# Лабораторная работа




In [ ]:
import warnings
import re

import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pickle
from tqdm.notebook import trange, tqdm
from catboost import CatBoostClassifier
from collections import Counter
from sentence_transformers import SentenceTransformer, util

## 1 EDA
Необходимо исследовать данные в предлагаемом Вам датасете. К исследованию данных относится анализ распределения значений в признаках, проверка скоррелированности признаков и т.д.

In [2]:
warnings.filterwarnings("ignore")

### 1.1. Загрузка данных

In [3]:
df = pd.read_csv('houses_ads_popularity.csv', index_col='Id', sep=',')

FileNotFoundError: [Errno 2] No such file or directory: 'houses_ads_popularity.csv'

### 1.2. Информация о DataFrame

#### 1.2.1. О столбцах и типах данных

In [ ]:
df.info()

#### 1.2.2. Описание данных         
* *bathrooms* - количество туалетных комнат      
* *bedrooms* - количество спальных комнат
* *building_id* - уникальный номер дома
* *created* - дата создания объявления 
* *description* - описание
* *display_address* - адрес расположения
* *features* - особенности
* *latitude* - широта
* *listing_id* - идентификатор
* *longitude* - долгота   
* *manager_id* - идентификатор менеджера
* *photos* - фотографии
* *price* - цена
* *street_address* - адрес по улице
* *TARGET* - популярность обьвления

#### 1.2.3. Первые 5 строк таблицы

In [ ]:
df.head()

### 1.3. Гистограммы для всех столбцов таблицы

In [ ]:
df.hist(figsize=(10, 10))
None

Рассмотрим гистограму столбца цены отдельно

In [ ]:
df['price'].hist(bins=100, range=[1, 10**4])
None

#### 1.3.1 Комментарий
* *bathrooms* - большинство квартир имеют 1 туалетную комнату, данные имеют распредение Пуассона
* *bedrooms* - распределение данных похоже на нормальное (судя по гистограмме)
* *latitude* - большинство квартир расположены на одной широте
* *longitude* - большинство квартир расположены на одной долготе
* *price* - имеет нормальное распределение, внутри диапазона цены (1, 10^4), с левосторонней ассиметрией. В данных присутствуют выбросы, далее обработем их.

### 1.4. Корреляция в данных

#### 1.4.1. Матрица корреляции

In [ ]:
corr_matrix = df.corr()
corr_matrix

#### 1.4.2. Тепловая карта

In [ ]:
sns.set(rc={'figure.figsize':(10, 10)})
sns.heatmap(corr_matrix, annot=True)
None

* *bathrooms* и *bedrooms* коррелируют, их коэфицент 0.53
* *longitude* и *latidude* линайно зависимы, коэфицент -0.99 

## 2 Feature engineering
Необходимо заполнить пропуски в данных, обработать категориальные признаки и т.д.

### 2.1. Обработка пропусков

#### 2.1.1. Количество пропусков

In [ ]:
df.isna().sum()

#### 2.1.2. Процент пропусков

In [ ]:
df.isna().sum() / df.shape[0] * 100

#### 2.1.3. Заполнение пропусков

In [ ]:
df['description'] = df['description'].str.strip()
df.loc[df['description'].isna() == True, 'description'] = 'None'
df.loc[df['description'] == '', 'description'] = 'None'

Пропусков незначительное количетво, заполним пропуски в столбце "description" константой "None", квартиры без адреса по улице удалим

In [ ]:
df.isna().sum().sum()

### 2.2. Изменение типов данных

#### 2.2.1. Численные

In [ ]:
df['TARGET'].value_counts()

In [ ]:
df['bathrooms'] = df['bathrooms'].astype('uint8')
df['bedrooms'] = df['bedrooms'].astype('uint8')

df['TARGET'] = df['TARGET'].replace('low', '0')
df['TARGET'] = df['TARGET'].replace('medium', '1')
df['TARGET'] = df['TARGET'].replace('high', '2')
df['TARGET'] = df['TARGET'].astype('uint8')

#### 2.2.2. Временные

In [ ]:
df['created'] = pd.to_datetime(df['created'])

#### 2.2.3. Категориальные

In [ ]:
gl_obj = df.select_dtypes(include='object').copy()
gl_obj.describe()

Что бы оптимизировать память и вычисления, необходимо категоризировать данные, в которых менее половины уникальных значений, в нашем случаи это 
* *building_id* 
* *display_address*
* *manager_id*
* *street_address*

In [ ]:
columns = ['building_id', 'display_address', 'manager_id', 'street_address']
for column in columns:
    labelencoder = LabelEncoder()
    data_new = labelencoder.fit_transform(df[column].values)
    df[column] = data_new
df.info()

#### 2.2.4. Фото
Укажем колличество фотографий у лбьявления (количество элементов в массиве)

In [ ]:
df.loc[df['photos'] == '[]', 'photos'] = ''
df_photos = df['photos'].apply(lambda x: len(x.split(',')) if x != '' else 0)
df['photos'] = df_photos

In [ ]:
df['photos'] = df['photos'].astype('uint8')

#### 2.2.5. Текстовые
Текстовые поля описаний, закодируем слова в эмбэдинги, подробное использование тектовых фич будет в разделе 6. NLP

In [ ]:
features = df['features'].str.lower().str.strip()
features = features.apply(lambda row: row[1: -1].replace("'", "").replace(',', ''))
features.head()

In [ ]:
df['features'] = features

Очистим текст от html тэгов

In [ ]:
def remove_html(row) -> list:
  row = re.sub(r'(\<(/?[^>]+)>)', '', row)
  return row

In [ ]:
df['description'] = df['description'].str.lower().str.strip()
df['description'] = df['description'].apply(remove_html)

### 2.3. Обработка дубликатов

In [ ]:
df.drop(columns=['description', 'features']).duplicated().sum()

Дубликаты не обнаружены

### 2.4. Добавление дополнительных фич

Количество слов в описании и количество слов в features

In [ ]:
df['description_count_words'] = df['description'].apply(lambda matrix: len(matrix))
df['features_count_words'] = df['features'].apply(lambda matrix: len(matrix))

df['features_count_words'] = df['features_count_words'].astype('uint16')
df['description_count_words'] = df['description_count_words'].astype('uint16')

## 3 **Выбор целевой метрики**
Необходимо выбрать метрику качества, которую вы будете оптимизировать. Для выбранной метрики необходимо написать пояснение. В этом задании нет единственного правильного ответа, однако за откровенно неадекватный выбор или слабую аргументацию будут снижаться баллы.


Самая популярная и самая не эффективная матрика качества для задач классификации это accuracy,
она имеет право на существование только в случаи работы с классами имеющими равное количество экземпляров.

Посмотрим на количество обьявлений (экземпляров) каждого класса

In [ ]:
df['TARGET'].value_counts()

Процентное соотношение от общего количества

In [ ]:
df['TARGET'].value_counts() / df.shape[0] * 100

Классы не равные, следовательно, мы не можем использовать accuracy (не очень то и хотелось),
рассмотрим другие метрики (имеющие право на существование):
* **Precision** - точность
* **Recall** - полнота

Они зависят, в отличие от accuracy, от соотношения классов и потому применимы в условиях несбалансированных выборок.
Будем использовать f1 меру (совокупность полноты и точности)

## 4 **Проведение экспериментов**
Необходимо попробовать разные модели, изученные в рамках курса и оптимизировать ими выбранную вами целевую метрику. Решение, состоящее только из grid-search по гиперпараметрам Catboost, получает 0 баллов.


В классах TARGET не равное количество элементов, будем обучаться на датасете с одинаковым количество объявлений каждого класса

PS После тестирование оказалось что в случаи обучения на датасете с неравным количество обьявлений каждого класса, результаты оказались лучше чем на равно классовом. 0.73 прости 0.54

In [ ]:
#count = df.loc[df['TARGET'] == 2].shape[0]

In [ ]:
#df_0 = df.loc[df['TARGET'] == 0][:count]
#df_1 = df.loc[df['TARGET'] == 1][:count]
#df_2 = df.loc[df['TARGET'] == 2]

In [ ]:
#assert df_0.shape[0] == df_1.shape[0] == count, 'не равные размерности'

In [ ]:
#normal_target_df = pd.concat([df_0, df_1, df_2])
normal_target_df = df

In [ ]:
df_to_train = normal_target_df[
    ['bathrooms', 'bedrooms', 'building_id', 'manager_id', 'photos', 
     'price', 'features_count_words', 'description_count_words']
]
df_to_train.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_to_train, normal_target_df['TARGET'], test_size=0.2, random_state=42)

Нормализация данных

In [ ]:
transformer = Normalizer().fit(X_train)
X_train_normalize = transformer.transform(X_train)

transformer = Normalizer().fit(X_test)
X_test_normalize = transformer.transform(X_test)

In [ ]:
def get_report_of_model(model, X_train, y_train, X_test) -> None:
    '''
    1) Обучаем полученную модель
    2) Делаем предсказание
    3) Считаем метрики
    '''
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred)
    print(report)

### 4.1. CatBoost

In [ ]:
!pip install catboost

In [ ]:
catboosting = CatBoostClassifier(metric_period=300)

#### 4.1.1. Ненормализованные данные

In [ ]:
get_report_of_model(catboosting, X_train, y_train, X_test)

#### 4.1.2. Нормализованные данные

In [ ]:
get_report_of_model(catboosting, X_train_normalize, y_train, X_test_normalize)

После опытов проведенных ниже, хочу отметить что наилучший результат показал катбуст,
Нормализованные данные в этом эксперименте показали результат хуже

### 4.2. Метод К-ближайших соседей

In [ ]:
nnn = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)

#### 4.2.1. Ненормализованные данные

In [ ]:
get_report_of_model(nnn, X_train, y_train, X_test)

#### 4.2.2. Нормализованные данные

In [ ]:
get_report_of_model(nnn, X_train_normalize, y_train, X_test_normalize)

### 4.3. Решающее дерево

In [ ]:
dtc = DecisionTreeClassifier(random_state=42)

#### 4.3.1. Ненормализованные данные

In [ ]:
get_report_of_model(dtc, X_train, y_train, X_test)

#### 4.3.2. Нормализованные данные

In [ ]:
get_report_of_model(dtc, X_train_normalize, y_train, X_test_normalize)

### 4.4. Наивный байесовский классификатор

In [ ]:
gnb = GaussianNB()

#### 4.4.1. Ненормализованные данные

In [ ]:
get_report_of_model(gnb, X_train, y_train, X_test)

#### 4.4.2. Нормализованные данные

In [ ]:
get_report_of_model(gnb, X_train_normalize, y_train, X_test_normalize)

Самые плохие результаты

### 4.5. Линейный дискриминантный анализ

In [ ]:
lda = LinearDiscriminantAnalysis()

#### 4.5.1. Ненормализованные данные

In [ ]:
get_report_of_model(lda, X_train, y_train, X_test)

#### 4.5.2. Нормализованные данные

In [ ]:
get_report_of_model(lda, X_train_normalize, y_train, X_test_normalize)

### 4.6. Метод опорных векторов

In [ ]:
svc = svm.SVC()

#### 4.6.1. Ненормализованные данные

In [ ]:
get_report_of_model(svc, X_train, y_train, X_test)

#### 4.6.2. Нормализованные данные

In [ ]:
get_report_of_model(svc, X_train_normalize, y_train, X_test_normalize)

### 4.7. Multi-layer Perceptron (MLP)

In [ ]:
mlp = MLPClassifier()

#### 4.7.1. Ненормализованные данные

In [ ]:
get_report_of_model(mlp, X_train, y_train, X_test)

#### 4.7.2. Нормализованные данные

In [ ]:
get_report_of_model(mlp, X_train_normalize, y_train, X_test_normalize)

Мы можем заметить что нормализованные данные в среднем показывают результаты такие же, как и нормализованные

### 4.7. Ансамбль деревьев
Наилучший результат показало решающее дерево, попробуем использовать ансамбль

#### 4.7.1. Подбор гипер параметров

In [ ]:
def plot_fitting_curve(parameter: str, values: list, X_train, X_test, y_train, y_test):
    train_curve = []
    test_curve = []
    for value in tqdm(values):
        model = DecisionTreeClassifier(**{parameter: value})
        model.fit(X_train, y_train)
        y_pred_train, y_pred_test = model.predict(X_train), model.predict(X_test)
        train_curve.append(f1_score(y_train, y_pred_train, average='weighted'))
        test_curve.append(f1_score(y_test, y_pred_test, average='weighted'))
    sns.lineplot(x=values, y=train_curve, label="Train").set_title(parameter)
    sns.lineplot(x=values, y=test_curve, label="Test")

##### 4.7.1.1. Максимальня глубина

In [ ]:
plot_fitting_curve('max_depth', np.arange(1, 20), X_train, X_test, y_train, y_test)

Судя по графику max_depth = 6

##### 4.7.1.2. Минимальное количество образцов, необходимое для разделен

In [ ]:
plot_fitting_curve('min_samples_split', np.arange(2, 800, 2), X_train, X_test, y_train, y_test)

Судя по графику min_samples_split = 600


##### 4.7.1.3. Минимальное количество образцов, необходимое для нахождения в листовом узле

In [ ]:
plot_fitting_curve('min_samples_leaf', np.arange(1, 400), X_train, X_test, y_train, y_test)

Судя по графику min_samples_leaf = 160

#### 4.7.2. Ансамблирование

In [ ]:
def plot_fitting_curve(parameter: str, values: list, X_train, X_test, y_train, y_test, params: dict):
    train_curve = []
    test_curve = []
    for value in tqdm(values):
        params.update({parameter: value})
        model = RandomForestClassifier(**params)
        model.fit(X_train, y_train)
        y_pred_train, y_pred_test = model.predict(X_train), model.predict(X_test)
        train_curve.append(f1_score(y_train, y_pred_train, average='weighted'))
        test_curve.append(f1_score(y_test, y_pred_test, average='weighted'))
    val, idx = min((val, idx) for (idx, val) in enumerate(test_curve))
    sns.lineplot(x=values, y=train_curve, label="Train").set_title(parameter)
    sns.lineplot(x=values, y=test_curve, label="Test")

##### 4.7.2.1. С использованием подобранных гиперпараметров

In [ ]:
params = {'max_depth':6, 'min_samples_split':600, 'min_samples_leaf':160}

plot_fitting_curve('n_estimators', np.arange(1, 200), X_train, X_test, y_train, y_test, params)

Результаты оказались хуже, чем у модели без подобранных гипер параметров

##### 4.7.2.2. Без использования подобранных гиперпараметров

In [ ]:
params = {}

plot_fitting_curve('n_estimators', np.arange(1, 20), X_train, X_test, y_train, y_test, params)

Судя по графику n_estimators = 30, f1=0.7, эти результаты получены при использовании данных для обучения с разных колличеством обьектов каждого класса

#### 4.7.2. Бустинг

In [ ]:
modelClf = GradientBoostingClassifier()

modelClf.fit(X_train, y_train)
print(modelClf.score(X_test, y_test))

## 5 **Анализ ошибок модели**
Необходимо взять несколько примеров из тестовой части датасета, на которых модель дала неправильный ответ, и попытаться проанализировать их, найти закономерности, полученные выводы проверить на других примерах.

### 5.1. Используемая модель

In [ ]:
model = RandomForestClassifier(n_estimators=30)

get_report_of_model(model, X_train, y_train, X_test)

Наилучшие результаты из всех эксперементов

### 5.2. ДатаФрэйм на котором модель давала не правильные ответы

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
error_matrix = []
for i in range(len(y_pred)):
    if y_pred[i] != y_test.to_list()[i]:
        row = [*list(X_test.to_numpy())[i], y_pred[i], y_test.to_list()[i]]
        error_matrix.append(row)

In [ ]:
error_df = pd.DataFrame(
    data=error_matrix,
    columns=['bathrooms', 'bedrooms', 'building_id', 'manager_id', 'photos', 
     'price', 'features_count_words', 'description_count_words', 'y_pred', 'y_test']
)

In [ ]:
error_df.head()

### 5.3. Корреляция

#### 5.3.1. Матрица корреляции

In [ ]:
corr_matrix = error_df.corr()
corr_matrix

#### 5.3.2. Тепловая карта

In [ ]:
sns.set(rc={'figure.figsize':(10, 10)})
sns.heatmap(corr_matrix, annot=True)
None

y_pred имеет коэффицент корреляции с price = -0.19 (наибольший), в то время как y_test с price = -0.018. можно сделать вывод что модель нашла зависимости между ценой и популярнстью на тренировочном датасете, но на тестовом этой зависимости нет. 

Так же относительно высокий коэффицент y_pred и building_id, но на тестовой выборке этой зависимости нет. 

## 6 NLP


### 6.1. Подготовим датасет

In [ ]:
df = df.drop_duplicates('description')

Удалим дубликаты

In [ ]:
df['description'].unique().shape

Получим текста (description)

In [ ]:
dd = Counter(df['description'].values)

arr = []
for k, v in dd.items():
    arr.append((k, v))
sort_arr = sorted(arr, key=lambda x: x[1], reverse=True)

desc = df['description'].values

### 6.2. Работа с нейросетью

#### 6.2.1. Сделаем эмбэдинги

In [ ]:
!pip install sentence-transformers

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
class DescDataset(Dataset):
    def __init__(self, embs, df):
        self.embs = embs
        self.df = df
    
    def __len__(self):
        return len(self.df)
      
    def __getitem__(self, idx):
        embs = self.embs[idx]
        trg = self.df['TARGET'].values[idx]
        return torch.FloatTensor(embs), torch.LongTensor([trg])

In [ ]:

train_df, test_df = train_test_split(df, test_size=0.2)

In [ ]:
device = torch.device('cuda')
model = model.to(device)

In [ ]:
all_embs = model.encode(df['description'].values)

Считать ранее сохраненные эмбэдинги

In [ ]:
#all_embs = pickle.load(open('embs1.pkl', 'rb'))

Сохранить эмбэдинги

In [ ]:
pickle.dump(all_embs, open('embs1.pkl', 'wb'))

In [ ]:
all_embs.shape

In [ ]:
train_dataset = DescDataset(all_embs, train_df)
test_dataset = DescDataset(all_embs, test_df)

In [ ]:
def make_weights_for_balanced_classes(images, nclasses):                        
    count = [0] * nclasses                                                      
    for item in images:                                                         
        count[item[1]] += 1                                                     
    weight_per_class = [0.] * nclasses                                      
    N = float(sum(count))                                                   
    for i in range(nclasses):                                                   
        weight_per_class[i] = N/float(count[i])                                 
    weight = [0] * len(images)                                              
    for idx, val in enumerate(images):                                          
        weight[idx] = weight_per_class[val[1]]                                  
    return weight         

Сохраним веса

In [ ]:
weights = make_weights_for_balanced_classes(train_dataset, 3)                                                                
weights = torch.DoubleTensor(weights)                                       
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

In [ ]:
trainloader = DataLoader(train_dataset, batch_size=256, sampler=sampler)
testloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

Самописная сеть классификатор

In [ ]:
head = nn.Sequential(
    nn.Linear(768, 1536),
    nn.BatchNorm1d(1536),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1536, 1536),
    nn.BatchNorm1d(1536),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1536, 1536),
    nn.BatchNorm1d(1536),
    
    nn.Linear(1536, 3),
).to(device)

class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.m = []
        sh = 768
        for i in range(50):
            self.m.append(nn.Sequential(
                nn.Linear(sh, sh * 2),
                nn.BatchNorm1d(sh * 2),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(sh * 2, sh - 15),
                nn.BatchNorm1d(sh - 15),
                nn.ReLU(),
                nn.Dropout(0.2),
            ))

            sh -= 15
        
        self.m = nn.Sequential(*self.m)
        self.l = nn.Linear(18, 3)
    
    def forward(self, x):
        x = self.m(x)
        x = self.l(x)
        return F.sigmoid(x)


In [ ]:
head = Model().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
opt = torch.optim.AdamW(head.parameters(), lr=1e-4)
shed = torch.optim.lr_scheduler.StepLR(opt, 10, 0.1)

#### 6.2.2. Обучение нейросети

In [ ]:
for epoch in trange(100):
    model.train()
    train_loss = 0
    for i, batch in enumerate(trainloader):
        x, y = map(lambda x: x.to(device), batch)
        
        opt.zero_grad()
        y_pred = head(x)
        loss = criterion(y_pred, y.squeeze(1))
        train_loss += loss.item()

        loss.backward()
        opt.step()
    
    train_loss /= len(trainloader)
    
    model.eval()
    test_loss = 0

    Y = []
    Y_PREDS = []


    with torch.no_grad():
        for i, batch in enumerate(testloader):
            x, y = map(lambda x: x.to(device), batch)
            
            y_pred = head(x)
            loss = criterion(y_pred, y.squeeze(1))

            test_loss += loss.item()


            Y.append(y)
            Y_PREDS.append(torch.argmax(torch.softmax(y_pred, dim=1), dim=1))


    test_loss /= len(testloader)
    Y = torch.cat(Y, dim=0)
    Y_PREDS = torch.cat(Y_PREDS, dim=0)

    #print(f'Epoch: {epoch} \t\t TrainLoss: {train_loss} \t\t TestLoss: {test_loss}')
    #print(classification_report(Y.cpu().detach().numpy(), Y_PREDS.cpu().detach().numpy()))

In [ ]:
Y = []
Y_PREDS = []

model.eval()
with torch.no_grad():
    for i, batch in tqdm(enumerate(testloader), total=len(testloader)):
        x, y = map(lambda x: x.to(device), batch)
        y_pred = head(x)
        Y.append(y)
        Y_PREDS.append(torch.argmax(torch.softmax(y_pred, dim=1), dim=1))

In [ ]:
Y = torch.cat(Y, dim=0)
Y_PREDS = torch.cat(Y_PREDS, dim=0)

In [ ]:
Y_PREDS.unique()

#### 6.2.3. Результаты обучения

In [ ]:
print(classification_report(Y.cpu().detach().numpy(), Y_PREDS.cpu().detach().numpy()))

### 6.3. Комментарий

Нейросеть переобучилась, результаты оказались хуже, у наилучшей модели (лес)